In [52]:
#cleaning data
import pandas as pd
import numpy as np
import janitor
from zipfile import ZipFile
import os
import io
import csv
import datetime





In [31]:
outer = "data/WedgeZipOfZips_small.zip"


In [65]:
#cleaning the data
def clean_data(df):

    #convert to datetime
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    
    #assign data types
    df = df.astype({
        'register_no': 'float',
        'emp_no': 'float',
        'trans_no': 'float',
        'upc': 'string',
        'description': 'string',
        'trans_type': 'string',
        'trans_subtype': 'string',
        'trans_status': 'string',
        'department': 'float',
        'quantity': 'float',
        'Scale': 'float',
        'cost': 'float',
        'unitPrice': 'float',
        'total': 'float',
        'regPrice': 'float',
        'altPrice': 'float',
        'tax': 'float',
        'taxexempt': 'float',
        'foodstamp': 'float',
        'wicable': 'float',
        'discount': 'float',
        'memDiscount': 'float',
        'discountable': 'float',
        'discounttype': 'float',
        'voided': 'float',
        'percentDiscount': 'float',
        'ItemQtty': 'float',
        'volDiscType': 'float',
        'volume': 'float',
        'VolSpecial': 'float',
        'mixMatch': 'float',
        'matched': 'float',
        'memType': 'boolean',
        'staff': 'boolean',
        'numflag': 'float',
        'itemstatus': 'float',
        'tenderstatus': 'float',
        'charflag': 'string',
        'varflag': 'float',
        'batchHeaderID': 'boolean',
        'local': 'float',
        'organic': 'float',
        'display': 'boolean',
        'receipt': 'float',
        'card_no': 'float',
        'store': 'float',
        'branch': 'float',
        'match_id': 'float',
        'trans_id': 'float'
})
    
#check for null values
    print(df.isnull().sum())




In [43]:
# Define your schema column names for files without headers
schema_columns = ['datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 
                  'trans_type', 'trans_subtype', 'trans_status', 'department', 'quantity', 
                  'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 
                  'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 
                  'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 
                  'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 
                  'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag', 'varflag', 
                  'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 
                  'store', 'branch', 'match_id', 'trans_id']



In [75]:
# Step 1: Open the ZIP file
with ZipFile(outer, 'r') as outer_zip:
    outer_files = outer_zip.namelist()

    # Step 2: Loop through the files in the outer ZIP
    for outer_file in outer_files:
        if outer_file.endswith('.zip'):  # Check if it's a ZIP file within the outer ZIP
            print(f"Found inner ZIP file: {outer_file}")

            # Step 3: Open the inner ZIP file
            with outer_zip.open(outer_file) as inner_zip_file:
                # Step 4: Read the inner ZIP file as a ZipFile object
                with ZipFile(inner_zip_file) as inner_zip:
                    inner_files = inner_zip.namelist()

                    for file in inner_files:
                        if file.endswith('.csv'):
                            print(f"Processing CSV file: {file}")
                            
                            # Step 3: Read the CSV file
                            with inner_zip.open(file) as csv_file:
                                csv_file = io.TextIOWrapper(csv_file, encoding="utf-8")

                                 # Step 4: Sniff the delimiter from the file
                                first_bytes = csv_file.read(1024)  # Read first few bytes to sniff delimiter
                                csv_file.seek(0)  # Reset file pointer to the beginning

                                sniffer = csv.Sniffer()
                                try:
                                    detected_dialect = sniffer.sniff(first_bytes,delimiters=[',',';','\t'])  # Sniff the delimiter
                                    delimiter = detected_dialect.delimiter
                                    print(f"Detected delimiter: {delimiter}")
                                except csv.Error:
                                    delimiter = ','  # Default to comma if delimiter detection fails
                                    print("Could not detect delimiter, using default ','")

                                
                                # Peek at the first line to see if it contains headers
                                first_line = csv_file.readline()
                                csv_file.seek(0)  # Reset the reader to the beginning

                                if 'register_no' and 'card_no' in first_line.lower():  # Check if it's a header by looking for known column names
                                    print(f"File {file} contains a header.")
                                    df = pd.read_csv(csv_file, delimiter=delimiter)  # Load with header
                                else:
                                    print(f"File {file} does not contain a header.")
                                    df = pd.read_csv(csv_file,delimiter=delimiter, header=None, names=schema_columns)  # Load without header, using schema columns

                                # Step 4: Process the DataFrame (e.g., cleaning)
                                #print data types
                                print(df.columns)

                                #clean the data
                                clean_data(df)

                


                                # You can apply cleaning functions here if needed, before writing the file back to CSV or uploading it.


Found inner ZIP file: transArchive_201001_201003_small.zip
Processing CSV file: transArchive_201001_201003_small.csv
Detected delimiter: ,
File transArchive_201001_201003_small.csv contains a header.
Index(['datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description',
       'trans_type', 'trans_subtype', 'trans_status', 'department', 'quantity',
       'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax',
       'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount',
       'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty',
       'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType',
       'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag', 'varflag',
       'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no',
       'store', 'branch', 'match_id', 'trans_id'],
      dtype='object')
datetime               0
register_no            0
emp_no                 0
trans_no               0
upc 

TypeError: Need to pass bool-like values